# Rekurencyjne Sieci Neuronowe (RNN)

Materiały w większości pochodzą od pracowników wydziału matematyki i informatyki.




### Importy i Utilsy  (odpalić i schować)

In [ ]:
# imports 
import torch
import os
import unicodedata
import string
import numpy as np
from typing import Tuple, Optional, List

from torch.nn.functional import cross_entropy

import matplotlib.pyplot as plt 
from sklearn.metrics import f1_score

from torch.utils.data import Dataset, DataLoader

all_letters = string.ascii_letters
n_letters = len(all_letters)


class ListDataset(Dataset):
    
    def __init__(self, data, targets):
        
        self.data = data
        self.targets = targets
        
    def __getitem__(self, ind):
        
        return self.data[ind], self.targets[ind]
    
    def __len__(self):
        return len(self.targets)

    
def unicode_to__ascii(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'
                                                                 and c in all_letters)
                   

def read_lines(filename: str) -> List[str]:
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicode_to__ascii(line) for line in lines]


def letter_to_index(letter: str) -> int:
    return all_letters.find(letter)


def line_to_tensor(line: str) -> torch.Tensor:
    tensor = torch.zeros(len(line), n_letters)
    for i, letter in enumerate(line):
        tensor[i][letter_to_index(letter)] = 1
    return tensor

In [ ]:

!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zip


--2022-12-11 11:33:09--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.226.52.36, 13.226.52.128, 13.226.52.90, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.226.52.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  15.3MB/s    in 0.2s    

2022-12-11 11:33:10 (15.3 MB/s) - ‘data.zip’ saved [2882130/2882130]

Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/names/Itali

In [ ]:
# NOTE: you can change the seed or remove it completely if you like
torch.manual_seed(1337)

data_dir = 'data/names'

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

data = []
targets = [] 
label_to_idx = {}

# read each natonality file and process data 
for label, file_name in enumerate(os.listdir(data_dir)):

    label_to_idx[label] = file_name.split('.')[0].lower()
    
    names = read_lines(os.path.join(data_dir, file_name))
    data += [line_to_tensor(name) for name in names]
    targets += len(names) * [label]

# split into train and test indices
test_frac = 0.1
n_test = int(test_frac * len(targets))
test_ind = np.random.choice(len(targets), size=n_test, replace=False)
train_ind = np.setdiff1d(np.arange(len(targets)), test_ind)

targets = torch.tensor(targets)
train_targets = targets[train_ind]

# calculate weights for BalancedSampler
uni, counts = np.unique(train_targets, return_counts=True)
weight_per_class = len(targets) / counts
weight = [weight_per_class[c] for c in train_targets]
# preapre the sampler
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights=weight, num_samples=len(weight)) 

train_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in train_ind], targets=train_targets)
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=1, sampler=sampler)

test_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in test_ind], targets=targets[test_ind])
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1)

In [ ]:
# check out the content of the dataset
for i, (x, y) in enumerate(train_loader):
    break

print("x.shape:", x.shape)
print("name: ", end="")
for letter_onehot in x[0]:
    print(all_letters[torch.argmax(letter_onehot)], end="")

print("\ny:", label_to_idx[y.item()])

x.shape: torch.Size([1, 15, 52])
name: Paraskevopoulos
y: greek


### RNN

<h4> Zadanie 1. </h4>

Zaimplementuj "zwykłą" sieć rekurencyjną. 
![rnn](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

Przyjmijmy, że stan ukryty ma wymiarowość $H$. Wtedy komórka powinna być warstwą liniową o postaci: $\tanh(W^T [x_t, h_{t-1}] + b)$, gdzie $x_t \in \mathbb{R}^{D}$ to wejście w kroku $t$, $h_{t-1} \in \mathbb{R}^{H}$ to stan ukryty z poprzedniego kroku a $W \in \mathbb{R}^{(H + D) \times H}$ i $b \in \mathbb{R}^H$ to parametry naszego modelu.

* W klasie `RNN` należy zainicjalizować potrzebne wagi oraz zaimplementować główną logikę dla pojedynczej chwili czasowej $x_t$
* Wyjście z sieci może mieć dowolny rozmiar, potrzebna jest również warstwa przekształacjąca $H$-wymiarowy stan ukryty na wyjście (o takiej wymiarowości ile mamy klas w naszym problemie). 
* W pętli uczenia należy dodać odpowiednie wywołanie sieci. HINT: pamiętać o iterowaniu po wymiarze "czasowym".


In [ ]:
class RNN(torch.nn.Module):
    
    def __init__(self, 
                 input_size: int,
                 hidden_size: int, 
                 output_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        :param output_size: int
            Desired dimensionality of the output vector
        """
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.input_to_hidden = torch.nn.Linear(input_size + hidden_size, hidden_size)
        
        self.hidden_to_output = torch.nn.Linear(hidden_size, output_size)


    
    # for the sake of simplicity a single forward will process only a single timestamp 
    def forward(self, 
                input: torch.tensor, 
                hidden: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor 
            Input tesnor for a single observation at timestep t
            shape [batch_size, input_size]
        :param hidden: torch.tensor
            Representation of the memory of the RNN from previous timestep
            shape [batch_size, hidden_size]
        """

        combined = torch.cat([input, hidden], dim=1) 
        hidden = self.input_to_hidden(combined)
        output =  self.hidden_to_output(hidden)

        return output, hidden
    
    def init_hidden(self, batch_size: int) -> torch.Tensor:
        """
        Returns initial value for the hidden state
        """
        return torch.zeros(batch_size, self.hidden_size, requires_grad=True)

### Pętla uczenia

In [ ]:
n_class = len(label_to_idx)

# initialize network and optimizer
rnn = RNN(n_letters, 256, n_class)
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.01)   

# we will train for only a single epoch 
epochs = 1


# main loop
for epoch in range(epochs):
    
    loss_buffer = []
    
    for i, (x, y) in enumerate(train_loader):  
        
        optimizer.zero_grad()
        # get initial hidden state
        hidden = rnn.init_hidden(x.shape[0])

        # get output for the sample, remember that we treat it as a sequence
        # so you need to iterate over the 2nd, time dimensiotn

        seq_len = x.shape[1]

        for ix in range(seq_len):       
          output, hidden = rnn(x[:,ix,:], hidden) 

        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()  
        
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []
    

# evaluate on the test set
with torch.no_grad():
    ps = []
    ys = []
    correct = 0
    for i, (x, y) in enumerate(test_loader):
        ys.append(y.numpy())

        hidden = rnn.init_hidden(x.shape[0])
        seq_len = x.shape[1]
        
        for ix in range(seq_len):        
          output, hidden = rnn(x[:,ix,:], hidden) 
        
        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.15, "You should get over 0.15 f1 score, try changing some hyperparams!"

Epoch: 0 Progress:  0% Loss: 2.869
Epoch: 0 Progress:  6% Loss: 2.863
Epoch: 0 Progress: 11% Loss: 2.767
Epoch: 0 Progress: 17% Loss: 2.486
Epoch: 0 Progress: 22% Loss: 2.221
Epoch: 0 Progress: 28% Loss: 2.071
Epoch: 0 Progress: 33% Loss: 1.926
Epoch: 0 Progress: 39% Loss: 1.904
Epoch: 0 Progress: 44% Loss: 1.839
Epoch: 0 Progress: 50% Loss: 1.793
Epoch: 0 Progress: 55% Loss: 1.803
Epoch: 0 Progress: 61% Loss: 1.731
Epoch: 0 Progress: 66% Loss: 1.716
Epoch: 0 Progress: 72% Loss: 1.726
Epoch: 0 Progress: 77% Loss: 1.759
Epoch: 0 Progress: 83% Loss: 1.611
Epoch: 0 Progress: 89% Loss: 1.666
Epoch: 0 Progress: 94% Loss: 1.710
Epoch: 0 Progress: 100% Loss: 1.736
Final F1 score: 0.17


<h4> Zadanie 2. </h4> 

Zaimplementuj funkcje `predict`, która przyjmuje nazwisko w postaci stringa oraz model RNN i wypisuje 3 najlepsze predykcje narodowości dla tego nazwiska razem z ich logitami.

**Hint**: Przyda się tutaj jedna z funkcji z pierwszej komórki notebooka.

In [ ]:
def predict(name: str, rnn: RNN):
    """Prints the name and model's top 3 predictions with scores"""

    hidden_state = rnn.init_hidden(1)
    tensorified = line_to_tensor(name).unsqueeze(1)


    for i in range(len(name)):
        input = tensorified[i]
        output, hidden = rnn(input, hidden_state)
        hidden_state = hidden


    top_logits, top_indices = output.topk(3, dim = 1) 
    top_logits = torch.squeeze(top_logits)
    top_indices = torch.squeeze(top_indices)


    for logit, index in zip(top_logits, top_indices):
        label = label_to_idx[index.item()]
        print(f"{label}:  {logit.item():.3f}")


    return None



In [ ]:
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]

for name in some_names:
    print(name)
    predict(name, rnn)

Satoshi
japanese:  3.679
polish:  2.260
arabic:  1.983
Jackson
scottish:  3.843
english:  1.783
greek:  1.703
Schmidhuber
german:  3.671
dutch:  2.541
english:  1.817
Hinton
scottish:  3.487
english:  2.109
russian:  1.444
Kowalski
polish:  5.247
russian:  4.052
czech:  3.765


<h4> Zadanie 3 </h4>

Ostatnim zadaniem jest implementacji komórki i sieci LSTM. 

![lstm](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

* W klasie `LSTMCell` ma znaleźć się główna logika LSTMa, czyli wszystkie wagi do stanów `hidden` i `cell` jak i bramek kontrolujących te stany. 
* W klasie `LSTM` powinno znaleźć się wywołanie komórki LSTM, HINT: poprzednio było w pętli uczenia, teraz przenisiemy to do klasy modelu.
* W pętli uczenia należy uzupełnić brakujące wywołania do uczenia i ewaluacji modelu.

Zdecydowanie polecam [materiały Chrisa Olaha](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) do zarówno zrozumienia jak i ściągi do wzorów.

Zadaniem jest osiągnięcie wartości `f1_score` lepszej niż na sieci RNN, przy prawidłowej implementacji nie powinno być z tym problemów używając podanych hiperparametrów. Dozwolona jest oczywiście zmiana `random seed`.

#### Komórka LSTM

In [ ]:
class LSTMCell(torch.nn.Module):

    def __init__(self, 
                 input_size: int, 
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """
        
        super(LSTMCell, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size

        # initialize LSTM weights 
        # NOTE: there are different approaches that are all correct 
        # (e.g. single matrix for all input opperations), you can pick
        # whichever you like for this task
    
        self.weights = {
            'xi': torch.nn.Parameter(torch.Tensor(input_size, hidden_size)),
            'xf': torch.nn.Parameter(torch.Tensor(input_size, hidden_size)),
            'xo': torch.nn.Parameter(torch.Tensor(input_size, hidden_size)),
            'xg': torch.nn.Parameter(torch.Tensor(input_size, hidden_size)),
            'hi': torch.nn.Parameter(torch.Tensor(hidden_size, hidden_size)),
            'hf': torch.nn.Parameter(torch.Tensor(hidden_size, hidden_size)),
            'ho': torch.nn.Parameter(torch.Tensor(hidden_size, hidden_size)),
            'hg': torch.nn.Parameter(torch.Tensor(hidden_size, hidden_size)),
        }
        

        #zdecydowałem się zaimplementować modyfikację LSTM z biasami
        self.biases = {
            'i': torch.nn.Parameter(torch.Tensor(hidden_size)),
            'f': torch.nn.Parameter(torch.Tensor(hidden_size)),
            'o': torch.nn.Parameter(torch.Tensor(hidden_size)),
            'g': torch.nn.Parameter(torch.Tensor(hidden_size)),
        }

        #initailizing with small values - didnt work with torch.zeros
        for key in self.weights:
          torch.nn.init.xavier_normal_(self.weights[key])
        for key in self.biases:
          self.biases[key] = torch.nn.Parameter(torch.randn(hidden_size) * 0.01)

        
    def forward(self, 
                input: torch.tensor, 
                states: Tuple[torch.tensor, torch.tensor]) -> Tuple[torch.tensor, torch.tensor]:
        
        hidden, cell = states

        # Compute input, forget, and output gates
        # then compute new cell state and hidden state
        # see http://colah.github.io/posts/2015-08-Understanding-LSTMs/   


        i = torch.sigmoid(torch.matmul(input, self.weights['xi']) + torch.matmul(hidden, self.weights['hi']) + self.biases['i'])
        f = torch.sigmoid(torch.matmul(input, self.weights['xf']) + torch.matmul(hidden, self.weights['hf'])  + self.biases['f'])
        o = torch.sigmoid(torch.matmul(input, self.weights['xo']) + torch.matmul(hidden, self.weights['ho']) + self.biases['o'])
        g = torch.tanh(torch.matmul(input, self.weights['xg']) + torch.matmul(hidden, self.weights['hg']) + self.biases['g'])

        cell = f * cell + i * g
        hidden = o * torch.tanh(cell)

        return hidden, cell

### Klasa modelu LSTM

In [ ]:
class LSTM(torch.nn.Module):

    def __init__(self, 
                 input_size: int, 
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """
        
        super(LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.cell = LSTMCell(input_size=self.input_size, hidden_size=self.hidden_size)   
             
    def forward(self, 
                input: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor 
            Input tesnor for a single observation at timestep t
            shape [batch_size, input_size]
        Returns Tuple of two torch.tensors, both of shape [seq_len, batch_size, hidden_size]
        """
        
        batch_size = input.shape[0]
        
        hidden, cell = self.init_hidden_cell(batch_size)
        
        hiddens = []
        cells = []
        
        # this time we will process the whole sequence in the forward method
        # as oppose to the previous exercise, remember to loop over the timesteps
        
        seq_len = input.shape[1]
        
        for ix in range(seq_len):
            hidden, cell = self.cell(input=input[:, ix, :], states=(hidden, cell))
            hiddens.append(hidden)
            cells.append(cell)


        return torch.stack(hiddens), torch.stack(cells)
    
    def init_hidden_cell(self, batch_size):
        """
        Returns initial value for the hidden and cell states
        """
        return (torch.zeros(batch_size, self.hidden_size, requires_grad=True), 
                torch.zeros(batch_size, self.hidden_size, requires_grad=True))

### Pętla uczenia

In [ ]:
from itertools import chain

torch.manual_seed(1337)

# build data loaders
train_loader = DataLoader(train_dataset, batch_size=1, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=1)

# initialize the lstm with an additional cliassifier layer at the top
lstm = LSTM(input_size=len(all_letters), hidden_size=128)

clf = torch.nn.Linear(in_features=128, out_features=len(label_to_idx))

# initialize a optimizer
params = chain(lstm.parameters(), clf.parameters())
optimizer = torch.optim.Adam(params, lr=0.015) 

# we will train for only a single epoch 
epoch = 1

# main loop
for epoch in range(epoch):
    
    loss_buffer = []
    
    for i, (x, y) in enumerate(train_loader):   
        
        optimizer.zero_grad()

        # don't forget about the classifier!
        hidden, state = lstm(x)
        last_hidden = hidden[-1]
        output = clf(last_hidden)
        

        # calucate the loss
        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()                                
        
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []

# evaluate on the test set
with torch.no_grad():
    
    ps = []
    ys = []
    for i, (x, y) in enumerate(test_loader): 
        ys.append(y.numpy())
        hidden, state = lstm(x)
        last_hidden = hidden[-1]
        output = clf(last_hidden)
        

        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.18, "You should get over 0.18 f1 score, try changing some hiperparams!"

Epoch: 0 Progress:  0% Loss: 2.866
Epoch: 0 Progress:  6% Loss: 2.486
Epoch: 0 Progress: 11% Loss: 2.196
Epoch: 0 Progress: 17% Loss: 2.045
Epoch: 0 Progress: 22% Loss: 1.999
Epoch: 0 Progress: 28% Loss: 1.862
Epoch: 0 Progress: 33% Loss: 1.848
Epoch: 0 Progress: 39% Loss: 1.825
Epoch: 0 Progress: 44% Loss: 1.837
Epoch: 0 Progress: 50% Loss: 1.831
Epoch: 0 Progress: 55% Loss: 1.803
Epoch: 0 Progress: 61% Loss: 1.773
Epoch: 0 Progress: 66% Loss: 1.715
Epoch: 0 Progress: 72% Loss: 1.735
Epoch: 0 Progress: 77% Loss: 1.772
Epoch: 0 Progress: 83% Loss: 1.774
Epoch: 0 Progress: 89% Loss: 1.705
Epoch: 0 Progress: 94% Loss: 1.705
Epoch: 0 Progress: 100% Loss: 1.658
Final F1 score: 0.21


Próbowałem zejśc z loss poniżej 1, ale nie ważne ile zmieniałem to jakoś ta moja implementacja się zacina przy 1,7-1,4.
Chociaż F!_score jest większy niż dla RNN.

<h4> Zadanie 4. </h4>

Zaimplementuj analogiczną do funkcji `predict` z zadania 2 dla modelu `lstm+clf`.


In [ ]:
def predict_lstm(name: str, lstm: LSTM, clf: torch.nn.Module):
    """Prints the name and model's top 3 predictions with scores"""
    tensorified = line_to_tensor(name)
    tensorified = torch.unsqueeze(tensorified, dim = 0)
    hiddens, cells = lstm(tensorified)
    logits = clf(hiddens[-1])

    top_logits, top_indices = logits.topk(3, dim = 1) 

    top_logits = torch.squeeze(top_logits, dim = 0)
    top_indices = torch.squeeze(top_indices, dim = 0)


    for logit, index in zip(top_logits, top_indices):
        label = label_to_idx[index.item()]
        print(f"{label}:  {logit.item():.3f}")


    return None

In [ ]:
# test your lstm predictor
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]
    
for name in some_names:
    print(name)
    predict_lstm(name, lstm, clf)

Satoshi
japanese:  2.567
arabic:  1.889
italian:  0.791
Jackson
scottish:  1.095
russian:  0.193
korean:  -0.031
Schmidhuber
german:  2.808
french:  1.340
arabic:  1.339
Hinton
scottish:  0.838
english:  0.520
russian:  0.438
Kowalski
polish:  2.795
japanese:  0.886
greek:  0.599


Niestety nie udało mi się zejść do tak dużych różnic pomiędzy pierwszym a drugim miejscem, ale przynajmniej jęzeyki na pierwszym miejscu się zgadzają